In [ ]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dropout, Dense, BatchNormalization, LSTM, Bidirectional

from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

from tensorflow.keras.utils import to_categorical

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

from imblearn.over_sampling import SMOTE



from google.colab import drive

# v3:jab, cross, left hook, right hook, left uppercut, right uppercut, left overhand, right overhand, left lowkick, right lowkick, left body kick, right body kick, guard, block left kick, block right kick, left head block, right head block, neautral


In [ ]:
import tensorflow as tf
print(tf.__version__)
#IMPORTANT!!! Check version compatability or it won't run locally!!!

2.17.0


In [ ]:
# Data
df = pd.read_csv('kickboxing_data.csv')

In [ ]:
# Features and labels
x = df[['Left Elbow Angle', 'Left Knee Angle', 'Left Shoulder Angle', 'Left Hip Angle', 'Right Elbow Angle', 'Right Knee Angle', 'Right Shoulder Angle', 'Right Hip Angle']].values
y = df['Movement'].values

# Print sample values
print(x[600])
print(y[1206])

# Normalise the features
scaler = MinMaxScaler()
x = scaler.fit_transform(x)



# Reshape for the LSTM model- IMPORTANT
x = x.reshape(x.shape[0], -1)

[180.45 174.9  269.46 184.81   0.   164.4    0.   177.58]
1


In [ ]:
# Strip unwanted characters from labels and convert to integer- whoopsie :(
y = np.array([val.strip(']') for val in y], dtype='int')

In [ ]:

# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=69)

# Balance the training data via SMOTE
smote = SMOTE(random_state=69)
x_train_resampled, y_train_resampled = smote.fit_resample(x_train, y_train)

# Normalise x_train and x_test post SMOTE
scaler = StandardScaler()
x_train_resampled = scaler.fit_transform(x_train_resampled)
x_test_scaled = scaler.transform(x_test)

In [ ]:
# Implement k-Nearest Neighbours (KNN)
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(x_train_resampled, y_train_resampled)

# KNN predictions on train and test sets
knn_predictions_train = knn.predict(x_train_resampled).reshape(-1, 1)
knn_predictions_test = knn.predict(x_test_scaled).reshape(-1, 1)

# Combine KNN predictions with original features
x_train_combined = np.concatenate([x_train_resampled, knn_predictions_train], axis=1)
x_test_combined = np.concatenate([x_test_scaled, knn_predictions_test], axis=1)


In [ ]:
# Convert y to categorical (one hot encoding)
y_train_resampled = to_categorical(y_train_resampled, num_classes=18) #Including 0
y_test = to_categorical(y_test, num_classes=18)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=9)
y_train = to_categorical(y_train, num_classes=18)
y_test = to_categorical(y_test, num_classes=18)
# 8 angles
print(x_train.shape)
# 18 Categories of movement including 0. 0-17.
print(y_train.shape)

(133720, 8)
(133720, 18)


In [ ]:
# Define improved modelarchitecture
model = Sequential([
    Input(shape=(x_train_combined.shape[1], 1)),  # Shape (timesteps,features)
    # LSTM's
    # First layer
    Bidirectional(LSTM(64, return_sequences=True)),
    BatchNormalization(),
    Dropout(0.3),

    # Second LSTM layer
    LSTM(128, return_sequences=True),
    BatchNormalization(),
    Dropout(0.3),

    # Third LSTM layer (final layer)
    LSTM(64, return_sequences=False),
    BatchNormalization(),
    Dropout(0.3),

    # Fully connected
    Dense(128, activation='relu', kernel_regularizer=l2(0.002)),
    BatchNormalization(),
    Dropout(0.5),

    Dense(64, activation='relu', kernel_regularizer=l2(0.002)),
    BatchNormalization(),
    Dropout(0.5),

    Dense(32, activation='relu', kernel_regularizer=l2(0.002)),
    BatchNormalization(),

    # Output layer
    Dense(18, activation='softmax') # 0-17, 0 inclusive
])


# Compilation
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Define callbacks
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, min_lr=0.00003)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model
model.fit(x_train_combined, y_train_resampled, epochs=100, batch_size=64, validation_data=(x_test_combined, y_test), callbacks=[reduce_lr, early_stopping])

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(x_test_combined, y_test, verbose=2)

print('\nTest accuracy:', test_acc)

# Save
model.save('kickboxing_classifier_improved_with_knn.keras')

Epoch 1/100
155/155 ━━━━━━━━━━━━━━━━━━━━ 10s 27ms/step - accuracy: 0.1606 - loss: 3.1878 - val_accuracy: 0.0964 - val_loss: 3.1551 - learning_rate: 0.0010
Epoch 2/100
155/155 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - accuracy: 0.4121 - loss: 2.1581 - val_accuracy: 0.3772 - val_loss: 2.2180 - learning_rate: 0.0010
Epoch 3/100
155/155 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step - accuracy: 0.5356 - loss: 1.7774 - val_accuracy: 0.6361 - val_loss: 1.5838 - learning_rate: 0.0010
Epoch 4/100
155/155 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - accuracy: 0.6062 - loss: 1.5590 - val_accuracy: 0.7474 - val_loss: 1.3655 - learning_rate: 0.0010
Epoch 5/100
155/155 ━━━━━━━━━━━━━━━━━━━━ 6s 21ms/step - accuracy: 0.6482 - loss: 1.4120 - val_accuracy: 0.5286 - val_loss: 1.6358 - learning_rate: 0.0010
Epoch 6/100
155/155 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - accuracy: 0.6805 - loss: 1.3002 - val_accuracy: 0.7534 - val_loss: 1.2609 - learning_rate: 0.0010
Epoch 7/100
155/155 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - accuracy: 0.7154 - 